In [24]:
import os
import pickle
import json
import torch
import torch.nn.functional as F
import re

In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model_dir = 'disaster_model'
# Load best model
model = torch.load( os.path.join(model_dir, 'model_full.pt'),
            map_location=device,
            weights_only=False)

def predict_disaster(text, model, word_to_idx, max_length=50):
    def clean_text(text):
        text = text.lower()
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        text = re.sub(r'@\w+|#\w+', '', text)
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    
    # Tokenize text
    def tokenize(text, word_to_idx, max_length=50):
        words = text.split()
        tokens = [word_to_idx.get(word, word_to_idx['<UNK>']) for word in words]
        
        # Pad or truncate to fixed length
        if len(tokens) < max_length:
            tokens = tokens + [word_to_idx['<PAD>']] * (max_length - len(tokens))
        else:
            tokens = tokens[:max_length]
        
        return tokens
    
    # Clean and tokenize text
    cleaned_text = clean_text(text)
    tokens = tokenize(cleaned_text, word_to_idx, max_length)
    
    # Convert to tensor and add batch dimension
    tokens_tensor = torch.tensor([tokens], dtype=torch.long).to(device)
    
    # Set model to evaluation mode
    model.eval()
    
    # Make prediction
    with torch.no_grad():
        # Forward pass with handling of bidirectional LSTM output
        lstm_out, _ = model.lstm(model.embedding(tokens_tensor))
        # Get the last output from both directions
        last_hidden = torch.cat((lstm_out[:, -1, :model.lstm.hidden_size], 
                               lstm_out[:, 0, model.lstm.hidden_size:]), dim=1)
        # Apply dropout and final linear layer
        outputs = model.fc(model.dropout(last_hidden))
        probabilities = F.softmax(outputs, dim=1)
        predicted_class = outputs.argmax(dim=1).item()
        confidence = probabilities[0][predicted_class].item()
    
    return {
        'text': text,
        'predicted_class': predicted_class,
        'disaster_type': disaster_types[predicted_class],
        'confidence': confidence * 100,
        'probabilities': {disaster_types[i]: prob.item() * 100 for i, prob in enumerate(probabilities[0])}
    }

Using device: cuda


In [ ]:
example_tweets = [
    "Earthquake just hit the city! Buildings are shaking and people are running outside.",
    "The river has overflowed and many streets are now underwater. #flooding",
    "Hurricane warning in effect for the coastal areas. Everyone please evacuate.",
    "Just saw a tornado touch down near the highway. It's moving east quickly.",
    "The wildfire is spreading rapidly due to strong winds. Fire crews are responding.",
    "Just had a great lunch at the new restaurant downtown. Would recommend!"
]

word_to_idx = {}
with open(os.path.join(model_dir, 'word_to_idx.pkl'), 'rb') as f:
    word_to_idx = pickle.load(f)
with open(os.path.join(model_dir, 'disaster_types.json'), 'r') as f:
    disaster_types = json.load(f)         
    

for tweet in example_tweets:
    result = predict_disaster(tweet, model, word_to_idx)
    print(f"Text: {result['text']}")
    print(f"Predicted: {result['disaster_type']} (Class {result['predicted_class']})")
    print(f"Confidence: {result['confidence']:.2f}%")
    print("Probabilities:")
    for disaster_type, prob in result['probabilities'].items():
        print(f"  {disaster_type}: {prob:.2f}%")
    print("-" * 80)

Text: Earthquake just hit the city! Buildings are shaking and people are running outside.
Predicted: Earthquake (Class 1)
Confidence: 99.95%
Probabilities:
  No Disaster: 0.01%
  Earthquake: 99.95%
  Flood: 0.00%
  Hurricane: 0.04%
  Tornado: 0.00%
  Wildfire: 0.00%
--------------------------------------------------------------------------------
Text: The river has overflowed and many streets are now underwater. #flooding
Predicted: Hurricane (Class 3)
Confidence: 46.27%
Probabilities:
  No Disaster: 10.87%
  Earthquake: 1.24%
  Flood: 39.37%
  Hurricane: 46.27%
  Tornado: 1.18%
  Wildfire: 1.08%
--------------------------------------------------------------------------------
Text: Hurricane warning in effect for the coastal areas. Everyone please evacuate.
Predicted: Hurricane (Class 3)
Confidence: 99.97%
Probabilities:
  No Disaster: 0.02%
  Earthquake: 0.00%
  Flood: 0.00%
  Hurricane: 99.97%
  Tornado: 0.00%
  Wildfire: 0.00%
--------------------------------------------------------